In [1]:
import cv2
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from glob import glob
from tqdm import tqdm


### Load and slice videos for training

In [2]:
dataset_path = 'PushUps'
filenames = glob(dataset_path + '\*.avi')

train = pd.DataFrame()
train['video_name'] = filenames
train.shape

(1, 1)

In [3]:
train_video_tag = []
for i in range(train.shape[0]):
    train_video_tag.append(train['video_name'][i].split('/')[0])

train['tag'] = train_video_tag
train.shape

(1, 2)

In [4]:
# storing the frames from training videos
for i in tqdm(range(train.shape[0])):
    count = 0
    videoFile = train['video_name'][i]
    cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % 2 == 0):
            # storing the frames in a new folder named train_1
            filename ='train_1/' + videoFile.split('\\')[1] +"_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.55it/s]


### Load OpenPose COCO model

In [88]:
protoFile = 'Model/pose_deploy_linevec.prototxt'
weightsFile = 'Model/pose_iter_440000.caffemodel'

In [2]:
# mpi_protoFile = 'Model/mpi/pose_deploy_linevec.prototxt'
mpi_protoFile = 'Model/mpi/pose_deploy_linevec_faster_4_stages.prototxt'
mpi_weightsFile = 'Model/mpi/pose_iter_160000.caffemodel'

In [3]:
net = cv2.dnn.readNetFromCaffe(mpi_protoFile,mpi_weightsFile)

In [4]:
dataset_frame_path = 'train_1'
frame_filenames = glob(dataset_frame_path + '\*.jpg')

trainFrame = pd.DataFrame()
trainFrame['frames'] = frame_filenames
trainFrame.shape

(28, 1)

In [5]:
inWidth = 368
inHeight = 368

imgPaths = []
for frame in trainFrame['frames']:
    img = cv2.imread(frame)
#     img = cv2.resize(img, (inWidth, inHeight))
    imgPaths.append(img)
    
# cv2.imshow('img', imgPaths[0])
# cv2.waitKey(3)

testimg = imgPaths[0]
frameWidth = testimg.shape[1]
frameHeight = testimg.shape[0]

In [6]:
# inBlob = cv2.dnn.blobFromImage(imgPaths[1], 1.0/255, (imgPaths[1].shape[1], imgPaths[1].shape[0]), (0,0,0), swapRB=False, crop=False)
inBlob = cv2.dnn.blobFromImage(testimg, 1.0/255, (368, 368), (0,0,0), swapRB=False, crop=False)
net.setInput(inBlob)
output = net.forward()

In [7]:
kwinName="Pose"
cv2.namedWindow(kwinName, cv2.WINDOW_AUTOSIZE)

In [8]:
# MPI
BODY_PARTS = {"Head": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
"LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
"RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "Chest": 14,
"Background": 15}

#COCO
# BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
#                    "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
#                    "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
#                    "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }

In [9]:
# MPI
POSE_PAIRS = [ ["Head", "Neck"], ["Neck", "RShoulder"], ["RShoulder", "RElbow"],
             ["RElbow", "RWrist"], ["Neck", "LShoulder"], ["LShoulder", "LElbow"],
             ["LElbow", "LWrist"], ["Neck", "Chest"], ["Chest", "RHip"], ["RHip", "RKnee"],
             ["RKnee", "RAnkle"], ["Chest", "LHip"], ["LHip", "LKnee"], ["LKnee", "LAnkle"] ]
#COCO
# POSE_PAIRS = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
#                    ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
#                    ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
#                    ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
#                    ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"] ]

In [10]:
points = []
for i in range(len(BODY_PARTS)):
    # Slice heatmap of corresponging body's part.
    heatMap = output[0, i, :, :]
 
    # Originally, we try to find all the local maximums. To simplify a sample
    # we just find a global one. However only a single pose at the same time
    # could be detected this way.
    _, conf, _, point = cv2.minMaxLoc(heatMap)
    x = (frameWidth * point[0]) / output.shape[3]
    y = (frameHeight * point[1]) / output.shape[2]
 
    # Add a point if it's confidence is higher than threshold.
    points.append((int(x), int(y)) if conf > 0.1 else None)

In [11]:
for pair in POSE_PAIRS:
    partFrom = pair[0]
    partTo = pair[1]
    assert(partFrom in BODY_PARTS)
    assert(partTo in BODY_PARTS)
 
    idFrom = BODY_PARTS[partFrom]
    idTo = BODY_PARTS[partTo]
    if points[idFrom] and points[idTo]:
        cv2.line(testimg, points[idFrom], points[idTo], (255, 74, 0), 3)
        cv2.ellipse(testimg, points[idFrom], (4, 4), 0, 0, 360, (255, 255, 255), cv2.FILLED)
        cv2.ellipse(testimg, points[idTo], (4, 4), 0, 0, 360, (255, 255, 255), cv2.FILLED)
        cv2.putText(testimg, str(idFrom), points[idFrom], cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255),2,cv2.LINE_AA)
        cv2.putText(testimg, str(idTo), points[idTo], cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255),2,cv2.LINE_AA)

In [12]:
# H = output.shape[2]
# W = output.shape[3]
# points = []

# for i in range(output.shape[1]):
#     probMap = output[0,i,:,:]
#     minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)
    
#     x = (frameHeight * point[0]) / W
#     y = (frameWidth * point[1]) / H
    
#     if prob > 0.6:
#         cv2.circle(imgPaths[1], (int(x), int(y)), 15, (0, 255, 255), thickness=1, lineType=cv2.FILLED)
#         cv2.putText(imgPaths[1], "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 0, 255), 3, lineType=cv2.LINE_AA)  
#         points.append((int(x), int(y)))
#     else:
#         points.append(None)
# cv2.imshow("Out", imgPaths[0])
# cv2.waitKey(0)
# cv2.destroyAllWindows()    

In [13]:
cv2.imshow(kwinName, testimg)
cv2.waitKey(0)
cv2.destroyAllWindows()